In [1]:
import pandas as pd 
import numpy as np 

#### 1. Evaluation Metrics ####

In [ ]:
## Input: y_pred, a list of length n with the predicted labels,
## y_true, a list of length n with the true labels

## Calculates the precision of the predicted labels
def get_precision(y_pred, y_true):
    truePos, falsePos = 0, 0
    for i,j in zip(y_pred, y_true):
        if i == 1 and j == 1:
            truePos += 1
        elif i == 1 and j == 0:
            falsePos += 1
            
    if truePos + falsePos == 0: #incase there are no true positives or false positives
        return 0
    
    precision = truePos/(truePos + falsePos)
    return precision
    
## Calculates the recall of the predicted labels
def get_recall(y_pred, y_true): #TP/(TP + FN)
    truePos, falseNeg = 0, 0
    
    for i,j in zip(y_pred, y_true):
        

    return recall

## Calculates the f-score of the predicted labels
def get_fscore(y_pred, y_true):
    ## YOUR CODE HERE...

    return fscore